# 使用PyNative模式调试

<a href="https://gitee.com/mindspore/docs/blob/master/tutorials/source_zh_cn/middleclass/pynative_mode_and_graph_mode/pynative_mode.ipynb" target="_blank"><img src="https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_source.png"></a>

## 概述

MindSpore支持两种运行模式，在调试或者运行方面做了不同的优化：

- PyNative模式：也称动态图模式，将神经网络中的各个算子逐一下发执行，方便用户编写和调试神经网络模型。
- Graph模式：也称静态图模式或者图模式，将神经网络模型编译成一整张图，然后下发执行。该模式利用图优化等技术提高运行性能，同时有助于规模部署和跨平台运行。

默认情况下，MindSpore处于PyNative模式，可以通过`context.set_context(mode=context.GRAPH_MODE)`切换为Graph模式；同样地，MindSpore处于Graph模式时，可以通过 `context.set_context(mode=context.PYNATIVE_MODE)`切换为PyNative模式。

PyNative模式下，支持执行单算子、普通函数和网络，以及单独求梯度的操作。下面将详细介绍使用方法和注意事项。

> PyNative模式下为了提升性能，算子在device上使用了异步执行方式，因此在算子执行错误的时候，错误信息可能会在程序执行到最后才显示。
>
> PyNative不支持summary功能，图模式summary相关算子不能使用。

## 执行单算子

执行单个算子，并打印相关结果，如下例所示。




In [1]:
import numpy as np

import mindspore.nn as nn
from mindspore import context, Tensor
from mindspore import ParameterTuple
from mindspore.common.initializer import TruncatedNormal
from mindspore.nn import Dense, WithLossCell, SoftmaxCrossEntropyWithLogits, Momentum
from mindspore.common.initializer import Normal
import mindspore.ops as ops
from mindspore import ms_function
from mindspore import dtype as mstype

# 切换为PyNative模式
context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU")

# 打印Cond2d算子的输出
conv = nn.Conv2d(3, 4, 3, bias_init='zeros')
input_data = Tensor(np.ones([1, 3, 5, 5]).astype(np.float32))
output = conv(input_data)
print(output.asnumpy())

[[[[-0.03753662 -0.04495239 -0.04495239 -0.04495239 -0.04495239]
   [-0.03619385 -0.05331421 -0.05331421 -0.05331421 -0.03625488]
   [-0.03619385 -0.05331421 -0.05331421 -0.05331421 -0.03625488]
   [-0.03619385 -0.05331421 -0.05331421 -0.05331421 -0.03625488]
   [-0.03457642 -0.06256104 -0.06256104 -0.06256104 -0.02789307]]

  [[ 0.02323914  0.0028038   0.0028038   0.0028038   0.02058411]
   [ 0.0146637   0.02206421  0.02206421  0.02206421  0.03007507]
   [ 0.0146637   0.02206421  0.02206421  0.02206421  0.03007507]
   [ 0.0146637   0.02206421  0.02206421  0.02206421  0.03007507]
   [ 0.02024841  0.04263306  0.04263306  0.04263306  0.02479553]]

  [[ 0.03041077  0.03619385  0.03619385  0.03619385  0.01067352]
   [ 0.02885437  0.05844116  0.05844116  0.05844116  0.04669189]
   [ 0.02885437  0.05844116  0.05844116  0.05844116  0.04669189]
   [ 0.02885437  0.05844116  0.05844116  0.05844116  0.04669189]
   [-0.00769424  0.03271484  0.03271484  0.03271484  0.05795288]]

  [[ 0.02745056  0.

## 执行普通函数

将若干算子组合成一个函数，然后直接通过函数调用的方式执行这些算子，并打印相关结果，如下例所示。

> 本例中`add`以普通PyNative方法执行

In [2]:
# 定义add函数
def add_func(x, y):
    z = ops.add(x, y)
    z = ops.add(z, x)
    return z

x = Tensor(np.ones([3, 3], dtype=np.float32))
y = Tensor(np.ones([3, 3], dtype=np.float32))
output = add_func(x, y)
print(output.asnumpy())

[[3. 3. 3.]
 [3. 3. 3.]
 [3. 3. 3.]]


### 提升PyNative性能

为了提高PyNative模式下的前向计算任务执行速度，MindSpore提供了Staging功能，该功能可以在PyNative模式下将Python函数或者Python类的方法编译成计算图，通过图优化等技术提高运行速度，是一种混合运行机制。Staging功能的使用通过`ms_function`装饰器达成，该装饰器会将将模块编译成计算图，在给定输入之后，以图的形式下发执行。如下例所示：

> 本例中`add`以Staging模式执行


In [3]:
# 导入ms_function
from mindspore import ms_function

# 仍设定为PyNative模式
context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU")

add = ops.Add()

# 使用装饰器编译计算图
@ms_function
def add_fn(x, y):
    res = add(x, y)
    return res

x = Tensor(np.ones([4, 4]).astype(np.float32))
y = Tensor(np.ones([4, 4]).astype(np.float32))
z = add_fn(x, y)
print(z.asnumpy())

[[2. 2. 2. 2.]
 [2. 2. 2. 2.]
 [2. 2. 2. 2.]
 [2. 2. 2. 2.]]


在加装了`ms_function`装饰器的函数中，如果包含不需要进行参数训练的算子（如`pooling`、`add`等算子），则这些算子可以在被装饰的函数中直接调用，如上例所示。如果被装饰的函数中包含了需要进行参数训练的算子（如`Convolution`、`BatchNorm`等算子），则这些算子必须在被装饰等函数之外完成实例化操作。

示例代码：


In [4]:
# Conv2d实例化操作
conv_obj = nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, stride=2, padding=0)
conv_obj.init_parameters_data()

@ms_function
def conv_fn(x):
    res = conv_obj(x)
    return res

input_data = np.random.randn(2, 3, 6, 6).astype(np.float32)
z = conv_fn(Tensor(input_data))
print(z.asnumpy())

[[[[ 0.01881409  0.07189941  0.01500702]
   [-0.02801514  0.04571533  0.02406311]
   [ 0.05444336 -0.02810669  0.01261902]]

  [[-0.04251099  0.06616211  0.04983521]
   [ 0.03497314  0.01404572 -0.04241943]
   [-0.04119873  0.03289795 -0.0024147 ]]

  [[-0.01972961 -0.02323914 -0.03945923]
   [-0.08062744  0.0531311  -0.00913239]
   [ 0.10827637  0.06359863 -0.01600647]]

  [[ 0.04614258 -0.03024292 -0.00562668]
   [-0.11395264 -0.03393555  0.08898926]
   [-0.00019276 -0.06091309 -0.03179932]]]


 [[[ 0.04873657  0.00447845 -0.01876831]
   [-0.08288574 -0.00782776 -0.02163696]
   [-0.01849365 -0.02975464  0.02043152]]

  [[ 0.03488159  0.02703857  0.05508423]
   [-0.04733276  0.03250122  0.0174408 ]
   [-0.05926514  0.05969238  0.06222534]]

  [[ 0.05154419 -0.01424408 -0.01612854]
   [-0.07617188 -0.00364304 -0.01573181]
   [-0.0748291   0.00495911 -0.04833984]]

  [[-0.04440308  0.01776123  0.02636719]
   [-0.00117493 -0.00774765  0.03512573]
   [-0.03546143  0.00117683  0.08581543]]

## 调试网络训练模型

PyNative模式下，还可以支持单独求梯度的操作。如下例所示，可通过`GradOperation`求该函数或者网络所有的输入梯度。需要注意，输入类型仅支持Tensor。


In [5]:
def mul(x, y):
    return x * y

def mainf(x, y):
    return ops.GradOperation(get_all=True)(mul)(x, y)

print(mainf(Tensor(1, mstype.int32), Tensor(2, mstype.int32)))

(Tensor(shape=[], dtype=Int32, value= 2), Tensor(shape=[], dtype=Int32, value= 1))


在进行网络训练时，求得梯度然后调用优化器对参数进行优化（暂不支持在反向计算梯度的过程中设置断点），然后再利用前向计算loss，从而实现在PyNative模式下进行网络训练。

下面在PyNative模式下进行LeNet训练：

1. 构建网络，该部分内容可以参考[快速入门-初学入门](https://www.mindspore.cn/tutorial/zh-CN/r1.2/quick_start.html)对应内容。


In [6]:
context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU")

class LeNet5(nn.Cell):
    """
    Lenet网络结构
    """
    def __init__(self, num_class=10, num_channel=1):
        super(LeNet5, self).__init__()
        # 定义所需要的运算
        self.conv1 = nn.Conv2d(num_channel, 6, 5, pad_mode='valid')
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode='valid')
        self.fc1 = nn.Dense(16 * 5 * 5, 120, weight_init=Normal(0.02))
        self.fc2 = nn.Dense(120, 84, weight_init=Normal(0.02))
        self.fc3 = nn.Dense(84, num_class, weight_init=Normal(0.02))
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def construct(self, x):
        # 使用定义好的运算构建前向网络
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# 实例化网络
net = LeNet5()

2. 如上文所说，利用`GradOperation`求函数的输入梯度。

In [7]:
class GradWrap(nn.Cell):
    """求函数输入梯度"""
    def __init__(self, network):
        super(GradWrap, self).__init__(auto_prefix=False)
        self.network = network
        # 用Tuple的形式包装weight
        self.weights = ParameterTuple(filter(lambda x: x.requires_grad, network.get_parameters()))

    def construct(self, x, label):
        weights = self.weights
        # 返回值为梯度
        return ops.GradOperation(get_by_list=True)(self.network, weights)(x, label)

3. 在PyNative模式中进行网络训练。

In [8]:
# 设定优化器、损失函数
optimizer = Momentum(filter(lambda x: x.requires_grad, net.get_parameters()), 0.1, 0.9)
criterion = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
# 通过WithLossCell获取Loss值
net_with_criterion = WithLossCell(net, criterion)
# 调用GradWrap
train_network = GradWrap(net_with_criterion)
train_network.set_train()

# 产生输入数据
input_data = Tensor(np.ones([32, 1, 32, 32]).astype(np.float32) * 0.01)
label = Tensor(np.ones([32]).astype(np.int32))
output = net(Tensor(input_data))

# 利用前向网络计算loss
loss_output = criterion(output, label)
# 求得梯度
grads = train_network(input_data, label)
# 优化参数
success = optimizer(grads)
loss = loss_output.asnumpy()
print(loss)

2.302585
